In [3]:
import ROOT
import sys
import numpy as np
import root_numpy as r_np

In [4]:
numch = 32
hEgam_gate_FA = [6637,6702] # 127I
hEgam_gate_FE = [6150,6194] # 127I
hEgam_gate_FA_bkg = [6777,7044] # 127I

In [30]:
def get_det_angle (file_name: str) -> list[int]:
    """Takes a string of a root file and returns a list containing the detector map"""

    file = ROOT.TFile(file_name)
    det_pos_graph = file.Get("det_pos_graph")
    det_pos = []
    pos_angle = [-1,71,90,109,109,90,71,90,-1,71,90,109,109,90,71,90,-1,36,72,108,144,-1,-1,-1,-1,144,108,72,36,-1,-1,-1] 
    det_angle = []

    for i in range(det_pos_graph.GetN()):
        det_pos.append(int(det_pos_graph.Eval(i)))

    file.Close()
    
    for i in range(numch):
        det_angle.append(pos_angle[det_pos[i]])

    return det_angle

def get_hist (file_name: str, hist_name: str) -> list[ROOT.TH1]:
    """Takes a string of a root file and a name of a histogram and returns a list of histograms"""

    hist = []
    file = ROOT.TFile(file_name)
    
    for i in range(numch):
        hist.append(file.Get(f"{hist_name}/{hist_name}_d{i}"))
        hist[i].SetDirectory(0)

    file.Close()
    
    return hist

In [6]:
def calc_scale(hEgam_bkg: list[ROOT.TH1]) -> list[float]:
    """Takes a list of histograms, calculates the scale and return the scales as a list"""

    scale = []

    for i in range(numch):
        try:     
            num = hEgam_bkg[i].Integral(hEgam_bkg[i].GetXaxis().FindBin(hEgam_gate_FA[0]),hEgam_bkg[i].GetXaxis().FindBin(hEgam_gate_FA[1]))
            dem = hEgam_bkg[i].Integral(hEgam_bkg[i].GetXaxis().FindBin(hEgam_gate_FA_bkg[0]),hEgam_bkg[i].GetXaxis().FindBin(hEgam_gate_FA_bkg[1]))
            scale.append(num/dem)
        except ZeroDivisionError:
            scale.append(0)
    
    return scale


def calc_hEn_sub(hist: list, hist_bkg: list, scale: list[float]) -> list[ROOT.TH1]:
    """Takes 2 lists of histograms and a list of scales and calculates a new histogram then returns the histograms as a list"""

    hist_bkg_sub = []
    for i in range(numch):
        hist_bkg_sub.append(hist[i]-hist_bkg[i]*scale[i])
        hist_bkg_sub[i].SetDirectory(0)

    return hist_bkg_sub

In [7]:
def get_A_LH_det(hist: list[ROOT.TH1],energy: float, width: float) -> tuple[list[float], list[float]]:
    """Takes a list of histograms and returns a list of A_LH_det and its error dA_LH_det"""
    
    N_L_det = []
    N_H_det = []
    A_LH_det = []
    dA_LH_det = []
    

    for i in range(numch):
        energy_bin = hist[i].FindBin(energy)
        width_bin = int(width//hist[i].GetBinWidth(energy_bin))
        N_L_det.append(hist[i].Integral(energy_bin - width_bin, energy_bin - 1))
        N_H_det.append(hist[i].Integral(energy_bin, energy_bin + width_bin))
    
        try:     
            A_LH_det.append(calc_A_LH(N_L_det[i],N_H_det[i]))
            dA_LH_det.append(calc_dA_LH(N_L_det[i],N_H_det[i]))
        except ZeroDivisionError:
            A_LH_det.append(None)
            dA_LH_det.append(None)

    return A_LH_det, dA_LH_det


def get_A_LH_angle (hist: list[ROOT.TH1], det_angle: list[int], energy: float, width: float) -> tuple[list[float], list[float]]:
    """Takes a list of histograms and returns a list of A_LH_det and its error dA_LH_det"""

    angle_list = [36,71,72,90,108,109,144]
    N_L_angle = [0] * len(angle_list)
    N_H_angle = [0] * len(angle_list)
    A_LH_angle = [] 
    dA_LH_angle = []

    for i in range(numch):
        energy_bin = hist[i].FindBin(energy)
        width_bin = int(width//hist[i].GetBinWidth(energy_bin))
        N_L = hist[i].Integral(energy_bin - width_bin, energy_bin - 1)
        N_H  = hist[i].Integral(energy_bin, energy_bin + width_bin)

        for j in range(len(angle_list)):
            if angle_list[j] == det_angle[i]:
                N_L_angle[j] = N_L_angle[j] + N_L
                N_H_angle[j] = N_H_angle[j] + N_H

    for j in range(len(angle_list)):
        try:     
            A_LH_angle.append(calc_A_LH(N_L_angle[j],N_H_angle[j])) 
            dA_LH_angle.append(calc_dA_LH(N_L_angle[j],N_H_angle[j]))
        except ZeroDivisionError:
            A_LH_angle.append(None)
            dA_LH_angle.append(None)

    return A_LH_angle, dA_LH_angle

In [29]:
def calc_N_LH(hist: list[ROOT.TH1],energy: float, width: float) -> tuple[list[float], list[float]]:
    """Takes a list of histograms and returns a list of N_L, N_H for each histogram"""
    
    N_L = []
    N_H = []
    

    for i in range(len(hist)):
        energy_bin = hist[i].FindBin(energy)
        width_bin = int(width//hist[i].GetBinWidth(energy_bin))
        N_L.append(hist[i].Integral(energy_bin - width_bin, energy_bin - 1))
        N_H.append(hist[i].Integral(energy_bin, energy_bin + width_bin))
    
        

    return N_L, N_H

In [33]:
# calc_N_LH(get_hist("stage0_output_NaI_1.root","hEn"),13.6,0.5)

In [42]:
def add_by_angle(list_by_det: list[list[any]], det_angle: list[list[int]]) -> list[any]:
    """Takes a lists of lists to be added by detector """
    
    angle_list = [36,71,72,90,108,109,144]
    list_by_angle = [0] * len(angle_list)
    
    for i, sublist in enumerate(list_by_det):
        for j, value in enumerate(sublist):
             for k, angle in enumerate(angle_list):
                if angle == det_angle[i][j]:
                    list_by_angle[k]+=value
               
    return list_by_angle

In [44]:
# list1 = [[0.01,0.01,0.01],[0.1,0.1,0.1],[1,1,1]]
# list2 = [[36,109,144],[36,109,144],[36,109,144]]
# add_by_angle(list1,list2)

In [28]:
def A_LH (N_L: float, N_H: float) -> float:
    """Takes two floats N_L and N_H and returns A_LH"""

    return (N_L - N_H) / (N_L + N_H)

def dA_LH (N_L: float, N_H: float) -> float:
    """Takes two floats N_L and N_H and returns dA_LH"""

    return 2 * ROOT.TMath.Sqrt((N_L * N_L * N_H + N_L * N_H * N_H)) / (N_L + N_H) / (N_L + N_H)

In [49]:
arr1 = np.array([1, 2, 3, -1, 5])
arr2 = np.array([2, 3, 4, 1, 6])
A_LH(arr1,arr2)

/var/folders/xg/rnpq_36x4jqgm4p1d_0gd47c0000gn/T/ipykernel_2993/2946472642.py:4: RuntimeWarning: divide by zero encountered in divide
  return (N_L - N_H) / (N_L + N_H)


array([-0.33333333, -0.2       , -0.14285714,        -inf, -0.09090909])

In [17]:
def draw_hist (hist: list[ROOT.TH1]) -> None:
    """Takes a list of histograms and draws each one"""

    global hist_canvas
    hist_canvas = []
    for i in range(len(hist)): 
        hist_canvas.append(ROOT.TCanvas(hist[i].GetName(),hist[i].GetName(),500,400))
        hist[i].GetXaxis().SetRangeUser(energy - 6 * width, energy + 6 * width)
        hist[i].Draw("E1")
        hist_canvas[i].Draw() 


In [25]:
input_file = "stage0_output_NaI_1.root"
E_p = 13.6
G_p = 0.5

det_angle = get_det_angle(input_file)

hEn = get_hist(input_file,"hEn_gated_FA")
hEn_bkg = get_hist(input_file,"hEn_gated_FA_bkg")
hEgam = get_hist(input_file,"hEgam")

scale = calc_scale(hEn_bkg)

hEn_sub = calc_hEn_sub(hEn, hEn_bkg, scale)

A_LH_det, dA_LH_det = get_A_LH_det(hEn_sub, E_p, G_p)
A_LH_angle, dA_LH_det = get_A_LH_angle(hEn_sub, det_angle, E_p, G_p)

In [34]:
# A_LH_det

[1, 2, 3, 3, 2, 1]